***
some experiments with roi geometries: geeutils.pixelcenterpoint and  geeutils.pixelinterspoint 
***

In [ ]:
import geemap
import ee
if not ee.data._credentials: ee.Initialize()

import sys
sys.path.append('../src/geepatches')
import geeutils



***
__pixelcenterpoint(eepoint, eerefimage)__ : center (point) of eerefimage pixel near eepoint\
__pixelinterspoint(eepoint, eerefimage)__ : intersection (point) of eerefimage pixels near eepoint
- __pixelcenterpoint__ + squarerasterboundsroi gives rois congruent with native raster for __odd__ diameters (in pixels)
- __pixelinterspoint__ + squarerasterboundsroi gives rois congruent with native raster for __even__ diameters (in pixels)
***

__Proba V 333m__

In [ ]:
eepoint = geeutils.tapspoint
eedate  = geeutils.fleecycloudsday
eeimage = geeutils.somePV333ndviImageNear(eedate, eepoint)

#print(geeutils.szestimatevaluesinfo(eeimage.clip(eepoint.buffer(10*333))))

eepixelcenterpoint = geeutils.pixelcenterpoint(eepoint, eeimage)

rois = [geeutils.squarerasterboundsroi(eepixelcenterpoint, pixelsradius, eeimage) for pixelsradius in [0.5, 1, 1.5, 2]]

map = geemap.Map(height='300px')
map.centerObject(eepixelcenterpoint, 14)
map.addLayer(eeimage, {'min':0, 'max':1}, 'eeimage')
for roi in rois:
    #print(geeutils.szgeometryinfo(roi))
    map.addLayer(geeutils.outlinegeometryimage(roi),                   {'palette':'#ff0000'})
map.addLayer(geeutils.outlinegeometryimage(eepoint,            10, 2), {'palette':'#00ff00'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eepixelcenterpoint, 10, 2), {'palette':'#0000ff'}, 'eepixelcenterpoint')
map


In [ ]:
eepoint = geeutils.tapspoint
eedate  = geeutils.fleecycloudsday
eeimage = geeutils.somePV333ndviImageNear(eedate, eepoint)

#print(geeutils.szestimatevaluesinfo(eeimage.clip(eepoint.buffer(10*333))))

eepixelinterspoint = geeutils.pixelinterspoint(eepoint, eeimage)
rois = [geeutils.squarerasterboundsroi(eepixelinterspoint, pixelsradius, eeimage) for pixelsradius in [0.5, 1, 1.5, 2]]

map = geemap.Map(height='300px')
map.centerObject(eepoint, 14)
map.addLayer(eeimage, {'min':0, 'max':1}, 'eeimage')
for roi in rois:
    #print(geeutils.szgeometryinfo(roi))
    map.addLayer(geeutils.outlinegeometryimage(roi),                   {'palette':'#ff0000'})
map.addLayer(geeutils.outlinegeometryimage(eepoint,            10, 2), {'palette':'#00ff00'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eepixelinterspoint, 10, 2), {'palette':'#0000ff'}, 'eepixelinterspoint')
map


__Sentinel 1 VV/VH bands__

In [ ]:
eepoint = geeutils.bobspoint
eedate  = geeutils.fleecycloudsday
eeimage = geeutils.someS1rviImageNear(eedate, eepoint)

print(geeutils.szestimatevaluesinfo(eeimage))

eepixelcenterpoint = geeutils.pixelcenterpoint(eepoint, eeimage)
rois = [geeutils.squarerasterboundsroi(eepixelcenterpoint, pixelsradius, eeimage) for pixelsradius in [1.5, 2, 2.5, 3, 3.5]]

map = geemap.Map(height='300px')
map.centerObject(eepoint, 18)
map.addLayer(eeimage, {'min':0, 'max':1}, 'eeimage')
for roi in rois:
    #print(geeutils.szgeometryinfo(roi))
    map.addLayer(geeutils.outlinegeometryimage(roi),                  {'palette':'#ff0000'})
map.addLayer(geeutils.outlinegeometryimage(eepoint,            1, 2), {'palette':'#00ff00'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eepixelcenterpoint, 1, 2), {'palette':'#ff0000'}, 'eepixelcenterpoint')
map

In [ ]:
eepoint = geeutils.bobspoint
eedate  = geeutils.fleecycloudsday
eeimage = geeutils.someS1rviImageNear(eedate, eepoint)

print(geeutils.szestimatevaluesinfo(eeimage))

eepixelinterspoint = geeutils.pixelinterspoint(eepoint, eeimage)
rois = [geeutils.squarerasterboundsroi(eepixelinterspoint, pixelsradius, eeimage) for pixelsradius in [1.5, 2, 2.5, 3, 3.5]]

map = geemap.Map(height='300px')
map.centerObject(eepoint, 18)
map.addLayer(eeimage, {'min':0, 'max':1}, 'eeimage')
for roi in rois:
    #print(geeutils.szgeometryinfo(roi))
    map.addLayer(geeutils.outlinegeometryimage(roi),                  {'palette':'#ff0000'})
map.addLayer(geeutils.outlinegeometryimage(eepoint,            1, 2), {'palette':'#00ff00'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eepixelinterspoint, 1, 2), {'palette':'#ff0000'}, 'eepixelinterspoint')
map

__Sentinel 1 'angle' band__
- using the COPERNICUS/S1_GRD_**FLOAT** collection
- compare angle bands between S1_GRD and S1_GRD_FLOAT
- no idea how S1_GRD.angle 'works'
- whatever I do (updateMask, clip, ...), a single pixel (roi) gives a gradient

In [ ]:
s1floatImageCollection    = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
s1floatrbgImageCollection = (s1floatImageCollection
                           .filter(ee.Filter.listContains('system:band_names','VV'))
                           .filter(ee.Filter.listContains('system:band_names','VH'))
                           .filter(ee.Filter.listContains('system:band_names','angle'))
                           .select(['VV', 'VH', 'angle']))                                  # False Color - not everywhere available

eepoint = geeutils.pastacosipoint
eedate  = geeutils.fleecycloudsday
eeimage = geeutils.someImageNear(s1floatrbgImageCollection.select(['angle']), eedate, eepoint)

print(geeutils.szestimatevaluesinfo(eeimage))

min = eeimage.reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
max = eeimage.reduceRegion(ee.Reducer.max()).values().get(0).getInfo()

eepixelcenterpoint = geeutils.pixelcenterpoint(eepoint, eeimage)
rois = [geeutils.squarerasterboundsroi(eepixelcenterpoint, pixelsradius, eeimage) for pixelsradius in [0.5, 1, 1.5, 2]]

map = geemap.Map(height='300px')
map.centerObject(eepoint, 8)
map.addLayer(eeimage, {'min':min, 'max':max}, 'eeimage')
for roi in rois:
    #print(geeutils.szgeometryinfo(roi))
    map.addLayer(geeutils.outlinegeometryimage(roi),                    {'palette':'#ff0000'})
map.addLayer(geeutils.outlinegeometryimage(eepoint,            300, 2), {'palette':'#00ff00'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eepixelcenterpoint, 300, 2), {'palette':'#ff0000'}, 'eepixelcenterpoint')
map

In [ ]:
s1floatImageCollection    = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
s1floatrbgImageCollection = (s1floatImageCollection
                           .filter(ee.Filter.listContains('system:band_names','VV'))
                           .filter(ee.Filter.listContains('system:band_names','VH'))
                           .filter(ee.Filter.listContains('system:band_names','angle'))
                           .select(['VV', 'VH', 'angle']))                                  # False Color - not everywhere available

eepoint = geeutils.pastacosipoint
eedate  = geeutils.fleecycloudsday
eeimage = geeutils.someImageNear(s1floatrbgImageCollection.select(['angle']), eedate, eepoint)

print(geeutils.szestimatevaluesinfo(eeimage))

min = eeimage.reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
max = eeimage.reduceRegion(ee.Reducer.max()).values().get(0).getInfo()

eepixelinterspoint = geeutils.pixelinterspoint(eepoint, eeimage)
rois = [geeutils.squarerasterboundsroi(eepixelinterspoint, pixelsradius, eeimage) for pixelsradius in [0.5, 1, 1.5, 2]]

map = geemap.Map(height='300px')
map.centerObject(eepoint, 8)
map.addLayer(eeimage, {'min':min, 'max':max}, 'eeimage')
for roi in rois:
    #print(geeutils.szgeometryinfo(roi))
    map.addLayer(geeutils.outlinegeometryimage(roi),                    {'palette':'#ff0000'})
map.addLayer(geeutils.outlinegeometryimage(eepoint,            300, 2), {'palette':'#00ff00'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eepixelinterspoint, 300, 2), {'palette':'#ff0000'}, 'eepixelinterspoint')
map

__Sentinel 2 radio bands__

In [ ]:
eepoint = geeutils.tapspoint
eedate  = geeutils.fleecycloudsday
eeimage = geeutils.someS2ndviImageNear(eedate, eepoint)

print(geeutils.szestimatevaluesinfo(eeimage))

eepixelcenterpoint = geeutils.pixelcenterpoint(eepoint, eeimage)
rois = [geeutils.squarerasterboundsroi(eepixelcenterpoint, pixelsradius, eeimage) for pixelsradius in [0.5, 1.5, 2.5, 9.5]]

map = geemap.Map(height='300px')
map.centerObject(eepoint, 17)
map.addLayer(eeimage, {'min':0, 'max':1}, 'eeimage')
for roi in rois:
    #print(geeutils.szgeometryinfo(roi))
    map.addLayer(geeutils.outlinegeometryimage(roi),                  {'palette':'#ff0000'})
map.addLayer(geeutils.outlinegeometryimage(eepoint,            2, 2), {'palette':'#00ff00'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eepixelcenterpoint, 2, 2), {'palette':'#ffffff'}, 'eepixelcenterpoint')
map

In [ ]:
eepoint = geeutils.tapspoint
eedate  = geeutils.fleecycloudsday
eeimage = geeutils.someS2ndviImageNear(eedate, eepoint)

print(geeutils.szestimatevaluesinfo(eeimage))

eepixelinterspoint = geeutils.pixelinterspoint(eepoint, eeimage)
rois = [geeutils.squarerasterboundsroi(eepixelinterspoint, pixelsradius, eeimage) for pixelsradius in [0.5, 1, 2, 9]]

map = geemap.Map(height='300px')
map.centerObject(eepoint, 17)
map.addLayer(eeimage, {'min':0, 'max':1}, 'eeimage')
for roi in rois:
    #print(geeutils.szgeometryinfo(roi))
    map.addLayer(geeutils.outlinegeometryimage(roi),                  {'palette':'#ff0000'})
map.addLayer(geeutils.outlinegeometryimage(eepoint,            2, 2), {'palette':'#00ff00'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eepixelinterspoint, 2, 2), {'palette':'#ffffff'}, 'eepixelinterspoint')
map